<i><b>Public AI</b></i>
<br>

# 멀티암드밴딧(Multi Armed Bandit)

지금까지 추천시스템의 다양한 모델을 다루어 보았고, 앞으로도 새로운 모델을 다루어 볼 것입니다. 비개인화 추천방식으로 가장 인기있는 Item을 선전하는 Top-K 추천모델과 유저와 아이템의 상호작용데이터를 활용한 협업필터링(CF)모델, 그리고 Implicit 데이터를 활용하기 위한 BPR 모델까지 다루었으며, 이제 앞으로 Metadata를 포함한 데이터를 위한 Factorization Machine이라는 모델도 배워볼 것입니다. 그렇다면, 상황에 따라 어떠한 모델을 활용하는 것이 좋을까요. 우리는 다양한 모델에 의한 추천이 제공되었을 때, 고객이 가장 잘 선택할 것 같은 추천 아이템을 한정해서 선택해야 합니다. 그리고 그 상황은 때때로 달라지곤 하지요. 이번 시간에는 각 시도에 따라 어떠한 모델의 추천이 가장 나을지 선택하는 Multi-Armed Bandit 에 대해 다루어 보겠습니다.

### _Objective_
1. **MAB 문제 이해하기** : 어떤 추천 모델이 고객에게 효과적인 추천 아이템을 선정하는지 확인할 수 있는 MAB 문제를 이해합니다

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import beta
from tensorflow.keras.utils import get_file

import matplotlib.pyplot as plt

# 문제 상황 : 어떤 추천 모형으로 고객에게 제공했을 때 클릭이 가장 많이 일어날까?
---

## 예제) 모형에 따라, 고객의 클릭 유무를 알려주는 클래스

<img src="https://imgur.com/3CSk6B5.png" width="500" > 

우리에게는 3가지 추천 모형이 있습니다. 세 추천 모형 모두 고객이 관심가질만한 제품을 노출시켜, 클릭을 유도하기 위해 설계되어 있습니다. 즉 높은 클릭율(CTR)일수록 좋은 모형입니다. 하지만 우리는 셋 중 어느 모형이 높은 CTR을 보일지 모릅니다. 이 때 우리는 어떻게 시도하는 것이 좋을까요?

In [ ]:
class Simulator:
    fpath = get_file(
        'MAB_simulation.csv',
        'https://pai-datasets.s3.ap-northeast-2.amazonaws.com/alai-deeplearning/MAB_simulation.csv')
    
    # ads_df : 각 시도에서 모델별 추천이 먹혔을 경우 1, 먹히지 않았을 경우 0인 dataframe
    ads_df = pd.read_csv(fpath)
    def __init__(self):
        self.idx = 0
        self.model_list = ['TOP_K','CF','BPR']
        
    def recommend_by(self, model_name):
        assert model_name in self.model_list
        self.idx += 1
        return bool(self.ads_df.loc[self.idx,model_name])

### 평가할 3가지 모형

* `Top-K` : 상위 K개의 아이템 노출
* `CF` : Item-Based Collaborative Filtering을 통한 아이템 노출
* `BPR` : Bayesian Personalized Ranking을 통한 아이템 노출

In [ ]:
simulator = Simulator()
simulator.model_list

### Case 1) Top-K을 이용해 추천했을 때의 결과



In [ ]:
for i in range(10):
    result = simulator.recommend_by('TOP_K')
    print("{}th 시도 : {}".format(i, result))

### Case 2) CF을 이용해 추천했을 때의 결과

In [ ]:
for i in range(10):
    result = simulator.recommend_by('CF')
    print("{}th 시도 : {}".format(i, result))

In [ ]:
cf_click = 0
for i in range(1000):
    cf_click += simulator.recommend_by('CF')
cf_click/1000

### Case 3) BPR을 이용해 추천했을 때의 결과

In [ ]:
for i in range(10):
    result = simulator.recommend_by('CF')
    print("{}th 시도 : {}".format(i, result))

### CTR을 알아내는 방법, 빈도 기반 확률 계산

각각 모델 별로 1000번씩 돌린다고 해봅시다.

In [ ]:
topk_click = 0
for i in range(1000):
    topk_click += simulator.recommend_by('TOP_K')
topk_click/1000

In [ ]:
cf_click = 0
for i in range(1000):
    cf_click += simulator.recommend_by('CF')
cf_click/1000

In [ ]:
bpr_click = 0
for i in range(1000):
    bpr_click += simulator.recommend_by('BPR')
bpr_click/1000

우리는 총 3000번의 시도를 통해, BPR 알고리즘의 CTR이 가장 높다는 것을 추론해냈습니다. 우리가 처음부터 BPR 알고리즘이 CTR이 가장 높았다는 것을 알았더라면, 800여번을 클릭 횟수를 이루어냈을텐데, 실험하는 과정에서 top_k와 CF 알고리즘으로 인해, 500여번의 훨씬 적은 클릭 횟수를 기록했습니다. 즉 우리는 300여번의 클릭을 놓친 셈이 됩니다. 이러한 것들은 기업의 입장에서는 모두 비용에 해당하게 됩니다. 어떻게 우리는 적은 시도 횟수로도 클릭을 이끌어낼 수 있을까요?

# [ Multi Armed Bandit을 푸는 알고리즘 , Thompson Sampling ]
---

## Thompson Sampling 알고리즘

### Beta 분포란?

베타 분포는 $\alpha$와 $\beta$라는 두 모수를 가지며 표본 공간은 0과 1 사이의 실수인 분포입니다.

$$
Beta(x;\alpha,\beta) = \frac{1}{B(\alpha,\beta)}x^{(\alpha-1)}(1-x)^{(\beta-1)}
$$

Beta 분포는 모수 $\alpha$,$\beta$를 통해 기댓값, 분산, 최빈값을 쉽게 추론할 수 있습니다.

* 기댓값 : $E[x] = \frac{\alpha}{\alpha+\beta}$
* 분산 : $Var[x] = \frac{\alpha\beta}{(\alpha+\beta)^2(\alpha+\beta+1)}$
* 최빈값 : $Mode[x] = \frac{\alpha-1}{\alpha+\beta-2}$

### Beta 분포를 통해 우리가 알 수 있는 것 -> 확률에 대한 확신 수준

10번 중 2번 고객이 클릭했을 때와 100번 중 20번 고객이 클릭했을 때 모두 CTR은 20%입니다. 그런데 10번을 했을 때 측정된 CTR이 믿을만할까요, 100번을 시도했을 때 측정된 CTR이 믿을만할까요? 베타 분포는 확신의 정도를 분포를 통해 보여줍니다.

#### case 1) 고객에게 10번 노출시켰을 때, 2번 클릭한 경우

In [ ]:
from scipy.stats import beta 

click = 2
total = 10

a = #
b = #

xs = np.linspace(0+1e-5,1-1e-5,100)
ys = #

ctr = #
var = #

plt.title("ctr : {:.3f}, var : {:.3f}".format(ctr,var))
plt.plot(xs,ys)
plt.show()

#### Case 2) 고객에게 100번 노출시켰을 때, 20번 클릭한 경우

In [ ]:
from scipy.stats import beta 

click = 20
total = 100

a = #
b = #

xs = np.linspace(0+1e-5,1-1e-5,100)
ys = #

ctr = #
var = #

plt.title("ctr : {:.1f}, var : {:.5f}".format(ctr,var))
plt.plot(xs,ys)
plt.show()

#### Case 3) 고객에게 1000번 노출시켰을 때, 200번 클릭한 경우

In [ ]:
from scipy.stats import beta 

click = 200
total = 1000

a = #
b = #

xs = np.linspace(0+1e-5,1-1e-5,100)
ys = #

ctr = #
var = #

plt.title("ctr : {:.1f}, var : {:.5f}".format(ctr,var))
plt.plot(xs,ys)
plt.show()

### Thompson Sampling을 통한 CTR 추정하기

In [ ]:
arms = pd.DataFrame(np.ones((3,2)), 
                    columns=['alpha','beta'],
                    index=['TOP_K','CF','BPR'])

clicks = 0
for idx in range(3000):
    # Visualization
    if idx % 300 == 0:
        for name, row in arms.iterrows():
            xs = np.linspace(0+1e-5,1-1e-5,100)
            ys = #

            plt.plot(xs,ys,label=name)
        plt.title(f'{idx}th attempt')
        plt.legend()
        plt.show()
        
    # Sampling Model
    bandit_result = #
    
    # Select 
    chosen_model = #  
    
    # Update Distribution
    click = #
    clicks += click
    if click:
        # Success
        arms.loc[chosen_model,'alpha'] += 1
    else:
        arms.loc[chosen_model,'beta'] += 1        

### Thompson Sampling을 통한 CTR 예측값

In [ ]:
#

<hr>
<div style = "background-image: url('https://pai-picture.s3.ap-northeast-2.amazonaws.com/PAI-Identity/PublicAI+Logo.png');background-repeat: no-repeat; background-position: right; background-size: 60px 40px; padding : 5px 70px 5px 5px;">
    Copyright(c) 2020 by Public AI. All rights reserved.<br>
    Writen by PAI, SeonYoul Choi ( best10@publicai.co.kr )  last updated on 2020/06/22
</div>
<hr>